This notebook analyses precipitation data to assess changes during typhoon season in Tokyo. It generates visualizations that illustrate projected changes in rainfall patterns under different climate scenarios for July, August, September and October, when typhoons are most common.

In [7]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import calendar
from matplotlib.ticker import MaxNLocator
import matplotlib.cm as cm
from pathlib import Path
import logging
import xarray as xr
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def load_datasets(data_paths):
    """
    Load CMIP6 datasets from given paths
    
    Parameters:
    -----------
    data_paths : list
        List of paths to NetCDF files
        
    Returns:
    --------
    dict : Dictionary of loaded datasets
    """
    datasets = {}
    
    for path in data_paths:
        try:
            # Extract scenario from filename
            path_obj = Path(path)
            filename = path_obj.name
            
            # Parse scenario from filename (assuming format cmip6_pr_SCENARIO_MODEL.nc)
            parts = filename.split('_')
            if len(parts) >= 3:
                scenario = parts[2]  # Get the scenario part
                model = parts[3].split('.')[0]  # Get model name without extension
                
                # Load dataset
                ds = xr.open_dataset(path)
                
                # Check if precipitation variable exists
                if 'pr' in ds:
                    # Convert from kg/m^2/s to mm/month if needed
                    if ds.pr.max() < 1:  # Likely in kg/m^2/s
                        # First convert to datetime to determine days in each month
                        time_data = pd.to_datetime(ds.time.values)
                        days_in_month = np.array([calendar.monthrange(dt.year, dt.month)[1] 
                                                for dt in time_data])
                        
                        # Create a DataArray with the same time dimension as pr
                        days_array = xr.DataArray(days_in_month, dims=['time'], coords={'time': ds.time})
                        
                        # Use xarray broadcasting to multiply correctly across dimensions
                        ds['pr'] = ds.pr * 86400 * days_array
                        logging.info(f"Converted precipitation from kg/m^2/s to mm/month")
                    
                    # Store in dictionary
                    if scenario not in datasets:
                        datasets[scenario] = {}
                    
                    datasets[scenario]['pr'] = ds.pr
                    logging.info(f"Loaded {scenario} data for model {model}")
                else:
                    logging.warning(f"No precipitation data found in {path}")
            else:
                logging.warning(f"Could not parse scenario from filename: {filename}")
                
        except Exception as e:
            logging.error(f"Error loading dataset {path}: {e}")
    
    return datasets

def extract_tokyo_data(datasets, tokyo_lat=35.6762, tokyo_lon=139.6503):
    """
    Extract data for Tokyo from datasets
    
    Parameters:
    -----------
    datasets : dict
        Dictionary of datasets by scenario
    tokyo_lat : float
        Latitude of Tokyo
    tokyo_lon : float
        Longitude of Tokyo
        
    Returns:
    --------
    dict : Dictionary of Tokyo data by scenario
    """
    tokyo_data = {}
    
    for scenario, data in datasets.items():
        if 'pr' in data:
            # Find closest grid point to Tokyo
            pr_data = data['pr']
            
            # Check if coordinates are 1D or 2D
            if len(pr_data.lat.shape) == 1 and len(pr_data.lon.shape) == 1:
                # For 1D coordinates
                lat_idx = abs(pr_data.lat - tokyo_lat).argmin().item()
                lon_idx = abs(pr_data.lon - tokyo_lon).argmin().item()
                
                # Extract data for Tokyo
                tokyo_pr = pr_data.isel(lat=lat_idx, lon=lon_idx)
            else:
                # For 2D coordinates, find the index of minimum distance
                lat_diff = abs(pr_data.lat - tokyo_lat)
                lon_diff = abs(pr_data.lon - tokyo_lon)
                total_diff = lat_diff + lon_diff
                min_idx = total_diff.argmin()
                
                # Convert to 2D indices
                lat_idx, lon_idx = np.unravel_index(min_idx, pr_data.lat.shape)
                
                # Extract data for Tokyo
                tokyo_pr = pr_data.isel(lat=lat_idx, lon=lon_idx)
            
            # Store in dictionary
            tokyo_data[scenario] = {'pr': tokyo_pr}
            
            logging.info(f"Extracted Tokyo data for {scenario}")
            logging.info(f"Time range: {tokyo_pr.time.min().values} to {tokyo_pr.time.max().values}")
    
    return tokyo_data

def analyze_typhoon_season(pr_data, time_periods=None):
    """
    Analyze precipitation during typhoon season (July-October)
    
    Parameters:
    -----------
    pr_data : xarray.DataArray
        Precipitation data for Tokyo (mm/month)
    time_periods : list of tuples, optional
        List of (start_year, end_year) tuples for analysis periods
        
    Returns:
    --------
    dict : Dictionary of typhoon season precipitation metrics
    """
    if time_periods is None:
        # Default: split into historical and future periods
        if pr_data.time.dt.year.min() < 2015:
            # For historical data
            time_periods = [(int(pr_data.time.dt.year.min().values), 2014)]
        else:
            # For future data
            time_periods = [(2015, int(pr_data.time.dt.year.max().values))]
    
    results = {}
    
    for start_year, end_year in time_periods:
        period_label = f"{start_year}-{end_year}"
        
        # Check if there is data for this period
        if start_year > pr_data.time.dt.year.max() or end_year < pr_data.time.dt.year.min():
            logging.warning(f"No data available for period {period_label}")
            continue
        
        period_data = pr_data.sel(time=slice(f"{start_year}", f"{end_year}"))
        
        if len(period_data) == 0:
            logging.warning(f"No data available for period {period_label}")
            continue
            
        # Create a pandas DataFrame
        df = period_data.to_dataframe(name='pr')
        df = df.reset_index()
        
        # Add date components
        df['year'] = df['time'].dt.year
        df['month'] = df['time'].dt.month
        df['day'] = df['time'].dt.day
        
        # Filter for typhoon season (July-October)
        typhoon_season = df[(df['month'] >= 7) & (df['month'] <= 10)]
        
        # Calculate metrics
        season_mean = typhoon_season.groupby('year')['pr'].mean()
        season_max = typhoon_season.groupby('year')['pr'].max()
        season_sum = typhoon_season.groupby('year')['pr'].sum()
        
        # Count extreme precipitation months during typhoon season
        # Define thresholds for different levels of extreme precipitation
        # These are adjusted from daily to monthly values (approximately 30x the daily values)
        extreme_thresholds = {
            'moderate': 900,  # mm/month (was 30 mm/day)
            'heavy': 1500,    # mm/month (was 50 mm/day) 
            'severe': 2400    # mm/month (was 80 mm/day)
        }
        
        extreme_days = {
            level: typhoon_season[typhoon_season['pr'] >= threshold].groupby('year').size()
            for level, threshold in extreme_thresholds.items()
        }
        
        # Calculate monthly distributions
        monthly_data = {}
        for month in range(7, 11):
            month_data = typhoon_season[typhoon_season['month'] == month]
            monthly_data[month] = {
                'mean': month_data.groupby('year')['pr'].mean(),
                'max': month_data.groupby('year')['pr'].max(),
                'sum': month_data.groupby('year')['pr'].sum()
            }
        
        # Calculate trends
        years = season_mean.index.values
        
        # Mean precipitation trend
        mean_slope, mean_intercept, mean_r, mean_p, mean_stderr = stats.linregress(years, season_mean)
        
        # Maximum precipitation trend
        max_slope, max_intercept, max_r, max_p, max_stderr = stats.linregress(years, season_max)
        
        # Total precipitation trend
        sum_slope, sum_intercept, sum_r, sum_p, sum_stderr = stats.linregress(years, season_sum)
        
        # Store results
        results[period_label] = {
            'season_mean': season_mean,
            'season_max': season_max,
            'season_sum': season_sum,
            'extreme_days': extreme_days,
            'monthly_data': monthly_data,
            'raw_data': typhoon_season,
            'mean_trend': {
                'slope': mean_slope,
                'intercept': mean_intercept,
                'p_value': mean_p,
                'r_value': mean_r
            },
            'max_trend': {
                'slope': max_slope,
                'intercept': max_intercept,
                'p_value': max_p,
                'r_value': max_r
            },
            'sum_trend': {
                'slope': sum_slope,
                'intercept': sum_intercept,
                'p_value': sum_p,
                'r_value': sum_r
            }
        }
        
    return results

def plot_scenario_comparison_key_metrics(tokyo_data, model_name, output_dir="./figures"):
    """
    Create comparison plots between historical and future scenarios for key metrics
    
    Parameters:
    -----------
    tokyo_data : dict
        Dictionary of Tokyo datasets by scenario
    model_name : str
        Name of the climate model
    output_dir : str
        Directory to save output figures
    """
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Setup figure aesthetics
    plt.style.use('seaborn-v0_8-whitegrid')
    sns.set_context("paper", font_scale=1.4)
    
    # Extract data for each scenario
    scenario_results = {}
    
    for scenario, data in tokyo_data.items():
        if 'pr' not in data:
            continue
            
        # Define time periods based on scenario
        if 'historical' in scenario:
            time_periods = [(int(data['pr'].time.dt.year.min().values), 2014)]
        else:
            time_periods = [(2015, int(data['pr'].time.dt.year.max().values))]
        
        # Analyze typhoon season
        results = analyze_typhoon_season(data['pr'], time_periods)
        
        # Store results
        scenario_results[scenario] = results
    
    # Create a figure for key metrics
    fig, axes = plt.subplots(2, 1, figsize=(12, 16))
    
    # Colors for scenarios
    colors = {
        'historical': 'darkblue',
        'ssp245': 'orange',
        'ssp585': 'darkred'
    }
    
    # Labels for scenarios
    labels = {
        'historical': 'Historical',
        'ssp245': 'SSP2-4.5 (Moderate)',
        'ssp585': 'SSP5-8.5 (High)'
    }
    
    # 1. Extreme Precipitation Months
    ax = axes[0]
    
    for scenario, results in scenario_results.items():
        # Process only the first time period for each scenario
        if not results:
            continue
            
        period = list(results.keys())[0]
        period_results = results[period]
        
        # Calculate average number of extreme months per year
        for threshold_name, threshold_data in period_results['extreme_days'].items():
            if threshold_name == 'heavy':  # Focus on heavy precipitation (e1500mm/month)
                avg_months = threshold_data.mean()
                years = len(threshold_data)
                
                # Plot as a horizontal bar
                ax.barh(
                    labels.get(scenario, scenario),
                    avg_months,
                    color=colors.get(scenario, 'gray'),
                    alpha=0.7,
                    xerr=threshold_data.std(),
                    capsize=5,
                    height=0.6
                )
                
                # Add the value at the end of each bar
                ax.text(
                    avg_months + 0.1,
                    labels.get(scenario, scenario),
                    f"{avg_months:.2f} months/year",
                    va='center'
                )
    
    ax.set_xlabel('Average Number of Heavy Precipitation Months (e1500mm) per Year')
    ax.set_title('Heavy Precipitation Months During Typhoon Season (Jul-Oct)')
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    
    # 2. Maximum Precipitation Intensity
    ax = axes[1]
    
    for scenario, results in scenario_results.items():
        # Process only the first time period for each scenario
        if not results:
            continue
            
        period = list(results.keys())[0]
        period_results = results[period]
        
        # Get maximum precipitation data and calculate statistics
        max_precip = period_results['season_max']
        avg_max = max_precip.mean()
        
        # Plot as a violin plot
        parts = ax.violinplot(
            max_precip.values,
            positions=[list(labels.values()).index(labels.get(scenario, scenario))],
            showmeans=False,
            showmedians=True,
            widths=0.8
        )
        
        # Color the violin plots
        for pc in parts['bodies']:
            pc.set_facecolor(colors.get(scenario, 'gray'))
            pc.set_alpha(0.7)
        
        # Add a marker for the mean
        ax.scatter(
            list(labels.values()).index(labels.get(scenario, scenario)),
            avg_max,
            color='black',
            marker='*',
            s=200,
            zorder=3
        )
        
        # Add text with the mean value
        ax.text(
            list(labels.values()).index(labels.get(scenario, scenario)),
            avg_max+12,
            f"Mean: {avg_max:.1f} mm",
            ha='center',
            fontsize=10
        )
    
    ax.set_ylabel('Maximum Monthly Precipitation (mm)')
    ax.set_title('Maximum Monthly Precipitation During Typhoon Season (Jul-Oct)')
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels.values())
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add a main title
    plt.suptitle(
        f'Typhoon Season Intensity Comparison - {model_name} Model',
        fontsize=16
    )
    
    # Adjust layout and save
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f"{output_dir}/{model_name}_key_metrics_comparison.png", dpi=300)
    plt.close()
    
    return fig

def plot_monthly_intensity_changes(tokyo_data, model_name, output_dir="./figures"):
    """
    Create a visualization showing changes in monthly precipitation intensity
    
    Parameters:
    -----------
    tokyo_data : dict
        Dictionary of Tokyo datasets by scenario
    model_name : str
        Name of the climate model
    output_dir : str
        Directory to save output figures
    """
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Setup figure aesthetics
    plt.style.use('seaborn-v0_8-whitegrid')
    sns.set_context("paper", font_scale=1.4)
    
    # Create figure
    fig, axes = plt.subplots(2, 2, figsize=(15, 12), sharey=True)
    axes = axes.flatten()
    
    # Month names
    month_names = {7: 'July', 8: 'August', 9: 'September', 10: 'October'}
    
    # Extract data for historical scenario
    historical_data = None
    future_data = {'ssp245': None, 'ssp585': None}
    
    for scenario, data in tokyo_data.items():
        if 'pr' not in data:
            continue
            
        # Convert to dataframe
        df = data['pr'].to_dataframe(name='pr').reset_index()
        df['year'] = df['time'].dt.year
        df['month'] = df['time'].dt.month
        
        # Filter for typhoon season
        typhoon_df = df[(df['month'] >= 7) & (df['month'] <= 10)]
        
        if 'historical' in scenario:
            historical_data = typhoon_df
        elif 'ssp245' in scenario:
            future_data['ssp245'] = typhoon_df
        elif 'ssp585' in scenario:
            future_data['ssp585'] = typhoon_df
    
    # If we don't have historical data, we can't proceed
    if historical_data is None:
        logging.warning("No historical data available for comparison")
        return None
    
    # Process data for each month
    for i, month in enumerate(range(7, 11)):
        ax = axes[i]
        
        # Extract historical data for this month
        hist_month = historical_data[historical_data['month'] == month]
        hist_values = hist_month['pr'].values
        
        # Create a dictionary to store future scenario data
        future_values = {}
        
        # Process each future scenario
        for scenario, future_df in future_data.items():
            if future_df is not None:
                # Extract data for this month
                future_month = future_df[future_df['month'] == month]
                future_values[scenario] = future_month['pr'].values
        
        # Create the plot
        # Historical data
        sns.kdeplot(
            hist_values,
            ax=ax,
            color='darkblue',
            label='Historical',
            fill=True,
            alpha=0.3
        )
        
        # Future scenarios
        for scenario, values in future_values.items():
            if scenario == 'ssp245':
                color = 'orange'
                label = 'SSP2-4.5 (Moderate)'
            else:
                color = 'darkred'
                label = 'SSP5-8.5 (High)'
                
            sns.kdeplot(
                values,
                ax=ax,
                color=color,
                label=label,
                fill=True,
                alpha=0.3
            )
        
        # Add median lines
        ax.axvline(
            np.median(hist_values),
            color='darkblue',
            linestyle='--',
            label='Historical Median'
        )
        
        for scenario, values in future_values.items():
            if scenario == 'ssp245':
                color = 'orange'
                label = 'SSP2-4.5 Median'
            else:
                color = 'darkred'
                label = 'SSP5-8.5 Median'
                
            ax.axvline(
                np.median(values),
                color=color,
                linestyle='--',
                label=label
            )
        
        # Calculate percent changes for annotation
        hist_median = np.median(hist_values)
        changes_text = []
        
        for scenario, values in future_values.items():
            future_median = np.median(values)
            percent_change = ((future_median - hist_median) / hist_median) * 100
            
            if scenario == 'ssp245':
                scenario_label = 'SSP2-4.5'
            else:
                scenario_label = 'SSP5-8.5'
                
            changes_text.append(f"{scenario_label}: {percent_change:.1f}% change")
        
        # Add the changes as text annotation
        ax.text(
            0.05, 0.95,
            '\n'.join(changes_text),
            transform=ax.transAxes,
            va='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8)
        )
        
        # Set titles and labels
        ax.set_title(f'{month_names[month]}')
        ax.set_xlabel('Monthly Precipitation (mm)')
        
        if i == 0 or i == 2:
            ax.set_ylabel('Density')
        
        # Add legend only to the first plot
        if i == 0:
            handles, labels = ax.get_legend_handles_labels()
            by_label = dict(zip(labels, handles))
            ax.legend(by_label.values(), by_label.keys(), loc='upper right')
        else:
            legend = ax.get_legend()
            if legend is not None:  # Only remove if legend exists
                legend.remove()
    
    # Add a main title
    plt.suptitle(
        f'Changes in Precipitation Distribution During Typhoon Season Months\n{model_name} Model',
        fontsize=16
    )
    
    # Adjust layout and save
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f"{output_dir}/{model_name}_monthly_intensity_changes.png", dpi=300)
    plt.close()
    
    return fig

def main():
    """Main function to run the analysis"""
    # Dataset paths
    data_paths = [
        "datasets/cmip6_pr_historical_MIROC6.nc",
        "datasets/cmip6_pr_ssp245_MIROC6.nc",
        "datasets/cmip6_pr_ssp585_MIROC6.nc"
    ]
    
    # Set model name
    model_name = "MIROC6"
    
    # Load datasets
    logging.info("Loading datasets...")
    datasets = load_datasets(data_paths)
    
    # Extract Tokyo data
    logging.info("Extracting Tokyo data...")
    tokyo_data = extract_tokyo_data(datasets)
    
    # Create output directory
    output_dir = "./figures"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Create key visualizations
    logging.info("Creating key metric comparisons...")
    fig1 = plot_scenario_comparison_key_metrics(tokyo_data, model_name, output_dir)
    
    logging.info("Creating monthly intensity change visualizations...")
    fig2 = plot_monthly_intensity_changes(tokyo_data, model_name, output_dir)
    
    logging.info("Analysis complete! Figures saved to: " + output_dir)
    
if __name__ == "__main__":
    main()

2025-04-24 12:18:47,778 - INFO - Loading datasets...
2025-04-24 12:18:49,152 - INFO - Converted precipitation from kg/m^2/s to mm/month
2025-04-24 12:18:49,153 - INFO - Loaded historical data for model MIROC6
2025-04-24 12:18:49,681 - INFO - Converted precipitation from kg/m^2/s to mm/month
2025-04-24 12:18:49,682 - INFO - Loaded ssp245 data for model MIROC6
2025-04-24 12:18:50,198 - INFO - Converted precipitation from kg/m^2/s to mm/month
2025-04-24 12:18:50,199 - INFO - Loaded ssp585 data for model MIROC6
2025-04-24 12:18:50,200 - INFO - Extracting Tokyo data...
2025-04-24 12:18:50,202 - INFO - Extracted Tokyo data for historical
2025-04-24 12:18:50,203 - INFO - Time range: 1850-01-16T12:00:00.000000000 to 2014-12-16T12:00:00.000000000
2025-04-24 12:18:50,203 - INFO - Extracted Tokyo data for ssp245
2025-04-24 12:18:50,204 - INFO - Time range: 2015-01-16T12:00:00.000000000 to 2100-12-16T12:00:00.000000000
2025-04-24 12:18:50,205 - INFO - Extracted Tokyo data for ssp585
2025-04-24 12: